In [1]:
import sifce
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical

In [2]:
# get ifos
ifos=['H1', 'L1', 'V1']
ifos_det = []

for ifo in ifos:
    ifos_det.append(Detector(ifo))

In [3]:
prior_gw150914 = dict(
                    mass_1= PowerLaw(alpha=-1, name='mass_1', minimum=10, maximum=80),
                    mass_ratio= PowerLaw(alpha=2, name='mass_ratio', minimum=0, maximum=1),
                    a_1 = Uniform(name='a_1', minimum=0, maximum=0.99),
                    a_2 = Uniform(name='a_2', minimum=0, maximum=0.99),
                    tilt_1 = Sine(name='tilt_1'),
                    tilt_2 = Sine(name='tilt_2'),
                    phi_12 = Uniform(name='phi_12', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    phi_jl = Uniform(name='phi_jl', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    theta_jn =  Sine(name='theta_jn'),
                    phase =  Uniform(name='phase', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    reference_frequency = DeltaFunction(20))

prior_sky = dict(dec=Cosine(name='dec'),
                ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),)

In [4]:
from sifce import population
from sifce import datatools

In [5]:
prior_obj = PriorDict(dictionary = prior_gw150914)
prior_sky_obj = PriorDict(dictionary = prior_sky)

pop = population.SimulationSet(distribution=prior_obj)
pop.sample_distribution(100)

sky_pop = prior_sky_obj.sample(10)
# print(pop.simulations_dataframe)

sky_pop

{'dec': array([-0.63246399,  0.83161579, -0.48753173,  0.31557314, -0.33528683,
        -0.14170237, -0.95862273, -0.0621921 ,  0.66833762,  0.26194348]),
 'ra': array([1.6802063 , 5.51939352, 5.79559205, 2.72036332, 3.27753333,
        0.22497571, 5.64897695, 0.24929285, 4.63362358, 0.64201673]),
 'psi': array([2.75167707, 1.54124933, 1.51497828, 0.19872762, 2.96883204,
        2.23715563, 2.51148296, 3.03548648, 1.69840543, 2.21289721])}

In [6]:
sample_rate=1024
flow = 20.0
delta_f = 1.0/8
flen = int(sample_rate/delta_f)
psd = analytical.aLIGOZeroDetHighPower(flen, delta_f, flow)

psd_dict = {'H1': psd, 'L1': psd, 'V1':psd}
apx='IMRPhenomXPHM'

In [12]:
import time
start_time = time.time()

sky_catter_results_list = list()

temp_pd = pop.simulations_dataframe
df = pd.DataFrame()
for index, row in temp_pd.iterrows():
    tmp_dict = row.to_dict()
    
    # get hp and hc:
    param_args = { k: tmp_dict[k] for k in pop.waveform_generation_labels}
    strain = dict()
    strain['plus'], strain['cross'] = datatools.compute_hphc_fd(param_args, apx)
    
    # projection into antenna:
    # position_args = { k: tmp_dict[k] for k in pop.detector_position_labels}
    projected_strains = dict()

    # for loop over sky positions:
    for i in range(len(sky_pop['ra'])):
        position_args = dict(ra = sky_pop['ra'][i], dec = sky_pop['dec'][i], psi = sky_pop['psi'][i])
        
        for ifo_name, ifo_detec in zip(ifos, ifos_det):
            projected_strains[ifo_name] = dict(template = datatools.project_and_combine(position_args, strain, 0, detector_obj=ifo_detec))

        # calculate snr
        opt_snr_dict, temp_snr_dict = datatools.compute_snr_fd(projected_strains, psd_dict)
        
        tmp = param_args | position_args
        tmp["opt_snr"] = opt_snr_dict['net']
        temp_df = pd.DataFrame(tmp, index=[0])
        df = pd.concat([df, temp_df], ignore_index=True)
        
    #sky scatter:
    opt_snr_arrays = datatools.sky_scatter_snr(sky_pop, {'plus_template': strain['plus'], 'cross_template': strain['cross']}, 0, psd_dict, ifo_list=ifos)
    sky_catter_results_list.append(opt_snr_arrays)
    
end_time = time.time()

In [19]:
(sky_catter_results_list[0][0]['net'])

array([ 9561.28792577, 12910.95820285, 10519.80353328, 11500.70053998,
       15195.39012397, 12462.68831551, 11738.76945047, 13643.85261413,
        9277.67430789, 12940.61885814])

In [28]:
tmp_snr_list = list()
for snr_tuple in sky_catter_results_list:
    tmp_snr_list.extend(snr_tuple[0]['net'])

In [29]:
df["opt_snr_sky_scatter"] = tmp_snr_list

In [32]:
pd.set_option("display.max_rows", None)
df[["opt_snr","opt_snr_sky_scatter"]]

,opt_snr,opt_snr_sky_scatter
0,9561.287871,9561.287926
1,12910.958138,12910.958203
2,10519.803470,10519.803533
3,11500.700485,11500.700540
4,15195.390056,15195.390124
5,12462.688266,12462.688316
6,11738.769435,11738.769450
7,13643.852571,13643.852614
8,9277.674258,9277.674308
9,12940.618787,12940.618858
